In [ ]:
# -*- coding: utf-8 -*-
# Copyright 2022 United Kingdom Research and Innovation
# Copyright 2022 The University of Manchester
# Copyright 2021 Technical University of Denmark
#
#   Authors:        Jakob S. Jørgensen (DTU)
#                   Edoardo Pasca (UKRI-STFC)
#                   Laura Murgatroyd (UKRI-STFC)
#                   Gemma Fardell (UKRI-STFC)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.



# Introductory demo of Core Imaging Library (CIL) 

### 3D laboratory micro-CT, cone-beam data of walnut

First import all modules we will need:

In [ ]:
import numpy as np
import os

from cil.io import TXRMDataReader, TIFFWriter
from cil.processors import TransmissionAbsorptionConverter, CentreOfRotationCorrector, Slicer
from cil.plugins.tigre import FBP
from cil.recon import FDK
from cil.utilities.display import show2D, show_geometry

Load the 3D cone-beam projection data of a walnut:

In [ ]:
base_dir = os.path.abspath("/mnt/materials/SIRF/Fully3D/CIL/")
data_name = "Walnut"
filename = os.path.join(base_dir, 
                        data_name, 
                        "valnut_2014-03-21_643_28",
                        "tomo-A",
                        "valnut_tomo-A.txrm")

data = TXRMDataReader(file_name=filename).read()

The data is loaded in as a CIL `AcquisitionData` object:

In [ ]:
type(data)

We can call `print` for the data to get some basic information:

In [ ]:
print(data)

Note how labels refer to the different dimensions. We infer that this data set contains 1601 projections each size 1024x1024 pixels.

In addition to the data itself, `AcquisionData` contains geometric metadata in an `AcquisitionGeometry` object in the `geometry` field, which can be printed for more detailed information:

In [ ]:
print(data.geometry)

CIL can illustrate the scan setup visually from the AcquisitionData geometry:

In [ ]:
show_geometry(data.geometry)

We can use the dimension labels to extract and display 2D slices of data, such as a single projection:

In [ ]:
show2D(data, slice_list=('angle',800))

From the background value of 1.0 we infer that the data is transmission data (it is known to be already centered and flat field corrected) so we just need to convert to absorption/apply the negative logarithm, which can be done using a CIL processor, which will handle small/large outliers:

In [ ]:
data = TransmissionAbsorptionConverter()(data)

We again take a look at a slice of the data, now a vertical one to see the central slice sinogram after negative logarithm:

In [ ]:
show2D(data, slice_list=('vertical',512))

CIL supports different back-ends for which data order conventions may differ. Here we use the CIL reconstructor class which provides the FDK algorithm with filtering using Intel Performance Primitives library and TIGRE backprojection.

We make sure that the data is in the correct order for TIGRE:

In [ ]:
data.reorder(order='tigre')

The last step to do is to correct for the centre of rotation misalignment. CIL offers 2 methods for finding the centre of rotation: [cross correlation](https://tomographicimaging.github.io/CIL/nightly/processors.html#cil.processors.CentreOfRotationCorrector.xcorrelation) and [image sharpness](https://tomographicimaging.github.io/CIL/nightly/processors.html#cil.processors.CentreOfRotationCorrector.image_sharpness). Here we will use image sharpness.

In [ ]:
cofr = CentreOfRotationCorrector.image_sharpness(FBP=FBP)
cofr(data, out=data)

We can now inspect the geometry and see where the centre of rotation is.

In [ ]:
print (data.geometry)

The data is now ready for reconstruction. To set up the FBP algorithm we must specify the size/geometry of the reconstruction volume. Here we use the default one:

In [ ]:
ig = data.geometry.get_ImageGeometry()

We can then create the FDK algorithm from CIL `recon` package, running on the GPU and reconstruct the data:

In [ ]:
fdk =  FDK(data, image_geometry=ig)
recon = fdk.run()

In [ ]:
show2D(recon, 
       slice_list=[('vertical', 512), 
                   ('horizontal_x', 512)], 
       fix_range=(-0.01,0.06))

We can save the reconstructed volume to disk for example as a stack of TIFFs:

In [ ]:
# save_base_path = os.getcwd()
# save_path = os.path.join(save_base_path, data_name)
# os.makedirs(save_path)

# TIFFWriter(data=recon, file_name=os.path.join(save_path, "out").write()

We now demonstrate the effect of reducing the number of projections on the FDK reconstruction.

In [ ]:
from cil.processors import Slicer

reduce_factor = 10

data_reduced = Slicer(
    roi={'angle': (0,-1,reduce_factor)}
)(data)

ig = data_reduced.geometry.get_ImageGeometry()
fbp =  FDK(data_reduced, image_geometry=ig)
recon_reduced = fbp.run()

We show the same slices as before:

In [ ]:
show2D(recon_reduced, 
       slice_list=[('vertical',512), ('horizontal_x', 512)], 
       fix_range=(-0.01,0.06))